<img src='https://www.actris.eu/sites/default/files/inline-images/Actris%20logo.png' width=200 align=right>

# ACTRIS DC 
## PM1 from wood burning and traffic


Aerosol Light Absorption Measurements can be used for quantitative determination of wood burning and traffic emission contributions to Carbonaceous Matter (CM). Based on the method described by Sandradewi et al., 2008, calculations of PM1 from fossil fuel (traffic) and wood burning (wb) can be derived from light absorption at 470 nm and 950 nm with light absorption exponents of 1.1 and 1.85 for traffic and wood burning, respectively. This method is mainly valid for Alpine regions in Europe, especially in winter.

The aerosol light absorption coefficients are measureded from filter absorption photometers, such as AE33 and CLAP. In the following examples it is demonstrated how Near Real Time (NRT) data from the ACTRIS thredds server are used to calculate and plot PM1 from wood burning and traffic. 

__Reference__:<br>
Sandradewi, J., Prevot, A. S. H., Szidat, S., Perron, N., Alfarra, M. R., Lanz, V. A., Weingartner, E., and Baltensperger, U.: Using aerosol light absorption measurements for the quantitative determination of wood burning and traffic emission contributions to particulate matter, Environ. Sci. Technol., 42, 3316–3323, https://doi.org/10.1021/es702253m, 2008

## Libraries 

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import threddsclient
#from processing import calc_daily_mean, calc_monthly_mean, calc_last3months_mean
import plotly.express as px
import re

The libraries "calc_daily_mean", "calc_monthly_mean" and "calc_last3months_mean" can be imported from a separate python program "processing.py", or they can be part the the main program. These libraries are used when we want to select the last row(s) of the array and write dates and values on a specific format.

In [2]:
import statistics
def calc_daily_mean(df_day):

    daily_mean = df_day.iloc[-1:].to_dict()

    # create variables for daily mean
    sample_date_daily_mean = list(daily_mean.keys())[0]
    sample_value_daily_mean = round(list(daily_mean.values())[0],4)

    return [sample_date_daily_mean, sample_value_daily_mean]

def calc_monthly_mean(df_month):

    monthly_mean = df_month.iloc[-1:].to_dict()
        
    # create variables for monthly mean    
    sample_date_monthly_mean = list(monthly_mean.keys())[0]
    sample_value_monthly_mean = round(list(monthly_mean.values())[0],4)

    return [sample_date_monthly_mean, sample_value_monthly_mean]

def calc_last3months_mean(df_month):

    # create date for last 3 months
    months = list(df_month.to_dict().keys())
    sample_date_last3months_mean = "{0} - {1}".format(months[-3].month_name(),months[-1].month_name())

    # create value (mean) for last 3 months
    last3months = statistics.mean(df_month.iloc[-3:].to_dict().values()) #include current month
    sample_value_last3months_mean = round(last3months,4)
    #last3months = statistics.mean(list(df_month.to_dict().values())[-3:-1]) #Do not include current month
    
    return [sample_date_last3months_mean, sample_value_last3months_mean]

## Read NRT data and plot time series
The program below will loop through all ACTRIS NRT 'filter_absorption_photometer' data files, extract the appropriate wavelengths, calculate PM1 from wood burning and traffic (plus total carbonaceous material), create html plots and print average PM1 values for the stations.

#### Equations and constants:

wavelength_traffic = 950.<br>
wavelength_wb = 470. <br>
$\alpha_{traffic}$=1.1 <br>
$\alpha_{wb}$=1.86 <br>

Trom Table 2:<br>
Traffic: $c1(1.1)=258831 ~(=0.258831*10^6)$ <br>
Wood: $c2(1.8)=601131,~ c2(1.9)=652516$ <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&rarr; $c2(1.86)=631962.4 ~(=0.6319624*10^6)$ <br>

Equations:<br>

(1) &nbsp;&nbsp;&nbsp;&nbsp;  $\frac{b_{abs}(470nm)_{traffic}}{b_{abs}(950nm)_{traffic}}=(\frac{470}{950})^{-\alpha_{traffic}} = ktr= kff$  

(2) &nbsp;&nbsp;&nbsp;&nbsp;  $\frac{b_{abs}(470nm)_{wb}}{b_{abs}(950nm)_{wb}}=(\frac{470}{950})^{-\alpha_{wb}} = kwb$  

(3a) &nbsp;&nbsp;&nbsp;  $b_{abs}(470nm)=b_{abs}(470nm)_{wb} + b_{abs}(470nm)_{traffic}$  
(3b) &nbsp;&nbsp;&nbsp;  $b_{abs}(950nm)=b_{abs}(950nm)_{wb} + b_{abs}(950nm)_{traffic}$  

Combination of Eq 1-3:

 $b_{abs}(950nm)_{wb}=(b_{abs}(470nm) - ktr*b_{abs}(950nm))/(kwb - ktr)$  
 $b_{abs}(470nm)_{wb}= kwb*b_{abs}(950nm)_{wb}$  
 $b_{abs}(950nm)_{traffic}= b_{abs}(950nm)-b_{abs}(950nm)_{wb}$  

Calcultate PM1:

(4) &nbsp;&nbsp;&nbsp;&nbsp;  $CM(PM1)=PM1_{traffic}+PM1_{wb}=c1*b_{abs}(950nm)_{traffic} + c2*b_{abs}(470nm)_{wb}$ 

 

In [5]:
#Define constants from Sandradewi et al., 2008:
given_wavelength_ff = 950. 
given_wavelength_wb = 470. 
alpha_ff=1.1
alpha_wb=1.86

#Table 2: c1=258831 (=0.258831E+6); c2(1.8)=601131, c2(1.9)=652516 --> c2(1.86)=631962.4
c1_ff=0.258331
c2_wb=0.631962

kff=(given_wavelength_wb/given_wavelength_ff)**(-alpha_ff)
kwb=(given_wavelength_wb/given_wavelength_ff)**(-alpha_wb)

#print(kwb)
#print(kff)

In [4]:
# Get the ACTRIS NRT thredds catalog
all_opendap_urls = threddsclient.opendap_urls('https://thredds.nilu.no/thredds/catalog/actris_nrt/catalog.xml')

## Get all filter absorption photometer opendap urls
#nrt_files = [f for f in all_opendap_urls if 'filter_absorption_photometer' in f]

# To select Italian stations (IT0009R) with filter absorption photometers
nrt_files = [f for f in all_opendap_urls if 'filter_absorption_photometer' in f and 'IT0009R' in f]
print('\n'.join(nrt_files))

for f in nrt_files:
    nrt_data = xr.open_dataset(f)
        
    #Read wavelengts in NRT file and select the two ones closest to 470n mand 950nm
    all_wavelengths = nrt_data.Wavelength.values 
    closest_wavelength_wb = min(all_wavelengths, key=lambda x:abs(x-given_wavelength_wb))
    closest_wavelength_ff = min(all_wavelengths, key=lambda x:abs(x-given_wavelength_ff))
    
    #Move to next NRT-file if wavelengts don't match
    if (np.abs(closest_wavelength_wb-given_wavelength_wb)>50) or (np.abs(closest_wavelength_ff-given_wavelength_ff)>50): continue   
   
    #Create data arrays with hourly aerosol absorption coefficients (arithmetic mean) for the two wavelengts (1/Mm)   
    ab470 = nrt_data.sel(Wavelength=closest_wavelength_wb).aerosol_absorption_coefficient_amean
    ab950 = nrt_data.sel(Wavelength=closest_wavelength_ff).aerosol_absorption_coefficient_amean

    #Calculate ab(950nm) for wood burning, based on Eq. 1-3 in Sandradewi. Neagative values are set to zero
    ab950wb = (ab470 - ab950*kff)/(kwb - kff)
    ab950wb[ab950wb < 0] = 0

    #Calculate PM1 from Eq. 1-4 in Sandradewi
    da_PM1_wb = (ab950wb * kwb * c2_wb).rename('PM1_wb')
    da_PM1_ff = ((ab950 - ab950wb) * c1_ff).rename('PM1_ff')

    # Convert DataArray (xarray) to pandas DataFrame, neagative values are set to zero
    df_wb = da_PM1_wb.to_dataframe()
    df_ff = da_PM1_ff.to_dataframe()
    df_wb[df_wb < 0] = 0
    df_ff[df_ff < 0] = 0

    #Create station name: a)selct the first part of the name, b)remove non-alphanumeric characters 
    station_string = nrt_data.ebas_station_name.split(" ",1)[0]
    station_name = re.sub('\W+','', station_string)

    # Create html plots of hourly data
    x=df_wb.index[:]
    df_hour_wb = df_wb.PM1_wb[:].rename('PM1_wood')
    df_hour_ff = df_ff.PM1_ff[:]
    df_hour_tot = df_hour_wb + df_hour_ff

    fig=px.line(df_hour_wb)
    fig.add_scatter(x=x,y=df_hour_ff, name = "PM1_traffic")
    fig.add_scatter(x=x,y=df_hour_tot, name="Carbonaceous mat.(PM1)")
    fig.update_layout(
        title="{0}".format(nrt_data.title),
        xaxis_title="Time",
        yaxis_title="µg/m3",
        font=dict(
            family="Courier New, monospace",
            size=18,
            color="RebeccaPurple"
        ),
    )
    #fig.write_html("timeseries_{0}.html".format(station_name.lower()))
    fig.show()



    #Calculate daily and monthly means (Pandas resample function)
    df_day_wb = df_wb.PM1_wb.resample('D').mean()
    df_day_ff = df_ff.PM1_ff.resample('D').mean()
    df_month_wb = df_wb.PM1_wb.resample('M').mean()
    df_month_ff = df_ff.PM1_ff.resample('M').mean()
   
    #Daily and monthly mean of total carbonaceous matter (PM1)
    df_day_tot = df_day_wb + df_day_ff
    df_month_tot = df_month_wb + df_month_ff


    # run mean functions and print values
    sample_date_daily_mean = calc_daily_mean(df_day_wb)[0]
    sample_value_daily_mean_wb = calc_daily_mean(df_day_wb)[1]
    sample_value_daily_mean_ff = calc_daily_mean(df_day_ff)[1]
    sample_value_daily_mean_tot = calc_daily_mean(df_day_tot)[1]

    sample_date_monthly_mean = calc_monthly_mean(df_month_wb)[0].month_name()
    sample_value_monthly_mean_wb = calc_monthly_mean(df_month_wb)[1]
    sample_value_monthly_mean_ff = calc_monthly_mean(df_month_ff)[1]
    sample_value_monthly_mean_tot = calc_monthly_mean(df_month_tot)[1]

    sample_date_last3months_mean = calc_last3months_mean(df_month_wb)[0]
    sample_value_last3months_mean_wb = calc_last3months_mean(df_month_wb)[1]
    sample_value_last3months_mean_ff = calc_last3months_mean(df_month_ff)[1]
    sample_value_last3months_mean_tot = calc_last3months_mean(df_month_tot)[1]
 

    print('PM1 monthly mean from',station_name,'in',sample_date_monthly_mean)
    print('Wood burning:',sample_value_monthly_mean_wb, 'ug/m3')
    print('Traffic:',sample_value_monthly_mean_ff, 'ug/m3')
    
print()
#print("Eosc-future example: https://folk.nilu.no/~toves/eosc/pm1_linux/station_monte.html")


https://thredds.nilu.no/thredds/dodsC/actris_nrt/IT0009R.20230622050000.20230922050411.filter_absorption_photometer...3mo.1h.IT06L_Magee_AE33_CMN_NRT.IT06L_AE33.lev1.5.nc


PM1 monthly mean from Monte in September
Wood burning: 0.6112 ug/m3
Traffic: 0.1565 ug/m3

